In [1]:
import numpy as np
import cv2
import open3d as o3d
from open3d import core as o3c

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [80]:
# Load calib results
proj_mtx =  np.load('../data/calib_results/bouda/proj_mtx.npy', )
proj_dist = np.load('../data/calib_results/bouda/proj_dist.npy', )
cam_mtx =   np.load('../data/calib_results/cam_1440/cam_mtx.npy', )
cam_dist =  np.load('../data/calib_results/cam_1440/cam_dist.npy')
proj_R =    np.load('../data/R.npy')
proj_T =    np.load('../data/T.npy')
R1 =        np.load('../data/R1.npy')
R2 =        np.load('../data/R2.npy')
P1 =        np.load('../data/P1.npy')
P2 =        np.load('../data/P2.npy')

proj_w, proj_h = (1280, 720)
calib_proj_w, calib_proj_h = (1920, 1080)

cam_w, cam_h = (2560, 1440)
scale_x = proj_w / calib_proj_w
scale_y = proj_h / calib_proj_h
proj_mtx[0, :] = proj_mtx[0, :] * scale_x
proj_mtx[1, :] = proj_mtx[1, :] * scale_y

base_path = '../data/recordings/elephant_800/'
h_pixels = np.load(base_path+'h_pixels.npy')
v_pixels = np.load(base_path+'v_pixels.npy')

In [81]:
cam_pts = []
proj_pts = []
colors = []

# Get white image to extract colors
img_white = cv2.imread(base_path + '/frame_1.jpg', cv2.IMREAD_COLOR)
img_white = cv2.cvtColor(img_white, cv2.COLOR_BGR2RGB)

for i in range(1000, 1700):
    for j in range(400, 1000):
# for i in range(cam_w):
#     for j in range(cam_h):
        h_value = h_pixels[j, i]
        v_value = v_pixels[j, i]
        if h_value == -1 or v_value == -1:
            pass
        else:
            cam_pts.append([i,j])
            h_value = min(proj_w-1, h_value)
            v_value = min(proj_h-1, v_value)
            proj_pts.append([ h_value, v_value])
            colors.append(img_white[j, i, :])

cam_pts = np.array(cam_pts, dtype=np.float32)
proj_pts = np.array(proj_pts, dtype=np.float32)

In [82]:
# newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mtx, cam_dist, (1920,1080), 1, (1920,1080))
# newprojmtx, roi = cv2.getOptimalNewCameraMatrix(proj_mtx, proj_dist, (1920,1080), 1, (1280,720))
# cam_pts_homo = cv2.undistortPoints(np.expand_dims(cam_pts, axis=1), cam_mtx, cam_dist, None, newcameramtx)
# proj_pts_homo = cv2.undistortPoints(np.expand_dims(proj_pts, axis=1), proj_mtx, proj_dist, None, newprojmtx)

cam_pts_homo = cv2.convertPointsToHomogeneous(cv2.undistortPoints(np.expand_dims(cam_pts, axis=1), cam_mtx, cam_dist, R=proj_R))[:,0].T
proj_pts_homo = cv2.convertPointsToHomogeneous(cv2.undistortPoints(np.expand_dims(proj_pts, axis=1), proj_mtx, proj_dist))[:,0].T
T = proj_T[:,0]

# Took from https://github.com/caoandong/Projector_Calibration/blob/master/visual_calib.py
TLen = np.linalg.norm(T)
NormedL = cam_pts_homo/np.linalg.norm(cam_pts_homo, axis=0)
alpha = np.arccos(np.dot(-T, NormedL)/TLen)
degalpha = alpha*180/np.pi
beta = np.arccos(np.dot(T, proj_pts_homo)/(TLen*np.linalg.norm(proj_pts_homo, axis=0)))
degbeta = beta*180/np.pi
gamma = np.pi - alpha - beta
P_len = TLen*np.sin(beta)/np.sin(gamma)
Pts = NormedL*P_len

colors_array = np.array(colors).astype(np.float64)/255.0

filter = (Pts[2] < 1.5) & (Pts[2] > -0.5)
Pts_filtered = Pts[:, filter]

pcd = o3d.t.geometry.PointCloud(o3c.Tensor(Pts[:, filter].T, o3c.float32))
pcd = pcd.to_legacy()
pcd.colors = o3d.cpu.pybind.utility.Vector3dVector(colors_array[filter])

o3d.visualization.draw_geometries([pcd])
# For bouda
# o3d.visualization.draw_geometries([pcd], zoom=0.45999999999999974,
#                                   front=[ -0.54775242945121028, -0.15624350092055739, -0.82192167780779746 ],
#                                   lookat=[ 0.30860175937414169, 0.14094289019703865, 0.47711528837680817 ],
#                                   up=[ 0.0089500924594270986, -0.9834453515238939, 0.18098380151552931 ])
